In [14]:
from __future__ import division
import os
import sys

from functools import partial
from glob import glob

import numpy as np
import pandas as pd
import scipy as sp

from Bio import SeqIO
from dendropy import Tree

from IPython.display import display
from ipywidgets import *

In [4]:
curDone = "chr9_23184280,chr13_66214808,chr10_90976645,chr8_61621505,chr10_74319510,chr8_14938556,chr2_179996168,chr8_31612815,chr8_84599283,chr21_20020024,chr19_463647,chr10_21782841,chr8_10937018,chr12_74034554,chr1_249122242,chr4_30875172,chr4_138877025,chr5_127674203,chr15_64179407,chr5_41645550,chr14_80084563,chr14_53295949,chr3_62686450,chr12_100739613,chr12_112865799,chr1_174017393,chr2_108791786,chr2_145688257,chr17_63251959,chr12_71956316,chr3_98057027,chr10_95102658,chr7_22639716,chr16_82851106,chr10_107092653,chr1_103725628,chr5_138033900,chr2_180426763,chr8_53132693,chr10_59764577,chr11_80940421,chr2_97646444,chr10_107406013,chr1_37751023,chr2_170302103,chr17_73242934,chr13_63785756,chr1_74882820,chr5_31425787,chr5_119682667,chr3_37162757,chr11_20487883,chr3_176493642,chr8_64691972,chr4_80310388,chr2_167947110,chr9_106793872,chr5_74675075,chr3_186874640,chr7_49236710,chr1_36496478,chr2_172221511,chr14_50414381,chr2_73114351,chr4_101983606,chr9_20252340,chr3_194172152,chr3_197206147,chr12_100520120,chr1_103618386,chr8_104662273,chr3_72805315,chr15_86486950,chr2_109443938,chr2_145673599,chr17_39706049,chr3_37175959,chr1_50960520,chr9_37820415,chr10_82209231,chr2_154775625,chr2_238372895,chr2_189597555,chr6_125859289,chr2_18403517,chr1_109697226,chr4_30425537,chr2_126174340,chr14_57692353,chr10_118693861,chr4_114957772,chr2_219305403,chr9_80256999,chr3_65429875,chr7_119447207,chr8_31750525,chr3_89604773,chr13_34925494,chr7_37110288,chr7_112411378,chr1_65124261,chr4_144903146,chr2_107809290,chr9_32488558,chr7_15582207,chr6_83322001,chr7_90737707,chr2_82545762,chr20_30562599,chr10_109831485,chr10_59607272,chr7_158036460,chr21_26964487,chr3_64527320,chr3_107036183,chr3_107504142,chr3_108741777,chr17_27778833,chr15_86888355,chr9_23185206,chr3_30874315,chr5_5604315,chr2_198057425,chr15_70762038,chr7_50823310,chr3_134032853,chr15_53521084,chr2_177612834,chr2_77455559,chr4_35973419,chr2_43699415,chr12_1381259,chr17_53802210,chr5_78012454,chr3_129142740,chr1_161988205,chr2_234513719,chr11_101093994,chr12_25720770,chr1_51106974,chr1_52016712,chr4_173549869,chr1_68391981,chr15_93577382,chr13_100802096,chr4_61620894,chr13_64236287,chr2_44162165,chr7_152729142,chr1_236026147,chr1_74357209,chr1_176752698,chr21_20039565,chr1_197034825,chr20_25421647,chr3_27100458,chr5_153110177,chr4_185616437,chr2_185696113,chr1_106963816,chr3_26471728,chr3_104783474,chr10_111775543,chr3_59921646,chr7_108862936,chr21_26977544,chr1_172922737,chr1_197165711,chr8_11450132,chr2_72426685,chr1_168657861,chr10_3049560,chr4_42019481,chr22_41849512,chr3_139229618,chr5_153676439,chr11_27924019,chr18_66709376,chr15_98856386,chr13_21820212,chr3_197715979,chr2_177625435,chr10_59722742,chr15_53140044,chr14_91177249,chr9_100596438,chr5_117494689,chr22_46731790,chr8_72741395,chr2_27406943,chr17_10466042,chr3_175271029,chr2_109345483,chr21_32415137,chr4_18680205,chr1_187501924,chr4_16801040,chr1_234315937,chr14_57675184,chr3_110735095,chr7_112745042,chr3_52387077,chr1_155382425,chr5_25415905,chr8_64626931,chr17_17733759,chr15_63886378,chr12_93117563,chr5_116951546,chr1_36028534,chr5_26637995,chr12_127024909,chr4_86470113,chr6_47765564,chr5_21824003,chr12_112493066,chr21_20013275,chr2_152585676,chr5_108096013,chr10_23864014,chr1_76223447,chr6_28173218,chr2_168356649,chr3_17810405,chr10_103340448,chr1_50862993,chr2_213537459,chr3_108764724,chr5_152917251,chr1_64530947,chr3_107876691,chr2_187717331,chr18_13666767,chr6_28427227,chr3_71492921"

In [15]:
# folders = map(partial(os.path.join,'/DATA/raw/dbpshp/EP'), curDone.split(','))
folders = glob('/DATA/raw/dbpshp/EP/*')
print len(folders)

24060


In [16]:
def increment():
    global pb,ph
    pb.value += 1
    ph.value = '{} / {}'.format(pb.value, pb.max)

In [17]:
pb = IntProgress(width='800px')
pb.value = 0
pb.max = len(folders)
ph = HTML('{} / {}'.format(pb.value, pb.max))
display(pb,ph)

for f in folders:
    rep = os.path.basename(f)
    csvs = sorted(glob(os.path.join(f,'AS/*.csv')))
#     trees = sorted(glob(os.path.join(f,'trees/*.nwk')))
    trees = [x.replace('AS/','trees/').replace('_AS.csv','.nwk') for x in csvs]
    
    ## Get norm times
    divTimes = []
    for t in trees:
        tree = Tree.get(path=t, schema='newick')
        root = tree.seed_node.edge
        if root.length is None:
            rel = 0
        else:
            rel = root.length
        l = tree.length() - rel
        divTimes.append(l)
    normTimes = np.array(divTimes)/sum(divTimes)
    
    data = []
    for i,csv in enumerate(csvs):
        df = pd.read_csv(csv, skiprows=2).set_index(' ')
        df2 = df.multiply(normTimes[i])
        data.append(df2)
    
    if len(data) == 0:
        increment()
        continue
    
    main = data[0]
    for df in data[1:]:
        main = main.add(df)
    main = main.divide(main.sum(axis=1), axis=0)
    
    main.to_csv(os.path.join(f,'{}.txt'.format(rep)))
    
    increment()